In [158]:
import sys
import json
import pickle
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas, pd_writer
import getpass as gt
import pandas as pd
import numpy as np

from snowflake_conn import *

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [2]:
conn = get_connection()

In [6]:
# members
sql = '''SELECT * FROM SANDBOX_CLAIMS.CMS_HCC._INT_MEMBERS;'''
mem = read_sql(sql)
mem = mem.iloc[:,:-3].sort_values(['patient_id']).reset_index(drop=True)

print(mem.shape)
mem.head()

(1000, 11)


,patient_id,enrollment_status,gender,age_group,medicaid_status,dual_status,orec,institutional_status,enrollment_status_default,medicaid_dual_status_default,institutional_status_default
0,10,New,Male,70-74,No,Non,Aged,No,True,True,True
1,10000,New,Female,66,No,Non,Aged,No,True,True,True
2,10002,New,Female,75-79,No,Non,Aged,No,False,True,True
3,10004,New,Female,80-84,No,Non,Aged,No,False,False,True
4,10007,New,Female,60-64,No,Non,Disabled,No,False,True,True


In [20]:
# hcc description
sql = '''SELECT DISTINCT hcc_code, description FROM SANDBOX_CLAIMS.CMS_HCC._value_set_disease_factors;'''
des = read_sql(sql)

d = dict(zip(des.hcc_code, des.description))
des.head()

,hcc_code,description
0,1,HIV/AIDS
1,2,"Septicemia, Sepsis, Systemic Inflammatory Resp..."
2,6,Opportunistic Infections
3,8,Metastatic Cancer and Acute Leukemia
4,9,Lung and Other Severe Cancers


In [21]:
# hcc -- most severe diagnosis when there is >1 diagnosis in the same group
sql = '''SELECT patient_id, hcc_code FROM SANDBOX_CLAIMS.CMS_HCC._INT_HCC_HIERARCHY;'''
hcc = read_sql(sql).sort_values(['patient_id', 'hcc_code']).reset_index(drop=True)
hcc['description'] = hcc['hcc_code'].map(d)

print(hcc.shape)
hcc.head()

(297, 3)


,patient_id,hcc_code,description
0,10002,85,Congestive Heart Failure
1,10004,108,Vascular Disease
2,10010,111,Chronic Obstructive Pulmonary Disease
3,10010,85,Congestive Heart Failure
4,10010,96,Specified Heart Arrhythmias


In [127]:
# risk scores
sql = '''
SELECT 
    patient_id,
    raw_risk_score,
    normalized_risk_score
FROM SANDBOX_CLAIMS.CMS_HCC.PATIENT_RISK_SCORES;
'''
risk = read_sql(sql).sort_values('patient_id').reset_index(drop=True)

print(risk.shape)
risk.head()

(1000, 3)


,patient_id,raw_risk_score,normalized_risk_score
0,10,0.786,0.757
1,10000,0.515,0.496
2,10002,0.860,0.829
3,10004,1.014,0.977
4,10007,0.000,0.000


In [130]:
# cost
sql = '''SELECT * FROM SANDBOX_CLAIMS.PMPM.PMPM_PREP;'''
pmpm = read_sql(sql)
pmpm = pmpm.iloc[:,:-1].sort_values('patient_id').reset_index(drop=True)

print(pmpm.shape)
pmpm.head()

(31184, 52)


,patient_id,year_month,inpatient_paid,outpatient_paid,office_visit_paid,ancillary_paid,pharmacy_paid,other_paid,acute_inpatient_paid,ambulance_paid,...,office_visit_allowed_2,outpatient_hospital_or_clinic_allowed,outpatient_psychiatric_allowed,outpatient_rehabilitation_allowed,skilled_nursing_allowed,urgent_care_allowed,total_paid,medical_paid,total_allowed,medical_allowed
0,10,201604,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,10,201704,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2,10,201610,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
3,10,201807,0.00,0.00,182.26,0.00,0.00,0.00,0.00,0.00,...,215.49,0.00,0.00,0.00,0.00,0.00,182.26,182.26,215.49,215.49
4,10,201711,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [35]:
# this currently is summing over multiple years
grouped = pmpm.groupby(by="patient_id")[['total_paid','medical_paid','total_allowed','medical_allowed']].sum().reset_index(drop=False)
grouped = grouped.sort_values('patient_id').reset_index(drop=True)

print(grouped.shape)
grouped.head()

(1000, 5)


,patient_id,total_paid,medical_paid,total_allowed,medical_allowed
0,10,4244.69,4244.69,3702.51,3702.51
1,10000,10453.12,10453.12,16507.85,16507.85
2,10002,6455.06,6455.06,6583.45,6583.45
3,10004,20881.06,20881.06,15210.91,15210.91
4,10007,28878.83,28878.83,16941.26,16941.26


In [91]:
# merge
df = mem.merge(hcc, on='patient_id', how="outer").merge(risk, on='patient_id', how="outer").merge(grouped, on='patient_id', how="outer")

cols = ['patient_id','hcc_code','raw_risk_score','normalized_risk_score','total_paid','medical_paid','total_allowed','medical_allowed']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

print(df.shape)
df.head()

(1127, 19)


,patient_id,enrollment_status,gender,age_group,medicaid_status,dual_status,orec,institutional_status,enrollment_status_default,medicaid_dual_status_default,institutional_status_default,hcc_code,description,raw_risk_score,normalized_risk_score,total_paid,medical_paid,total_allowed,medical_allowed
0,10,New,Male,70-74,No,Non,Aged,No,True,True,True,NaN,NaN,0.786,0.757,4244.69,4244.69,3702.51,3702.51
1,10000,New,Female,66,No,Non,Aged,No,True,True,True,NaN,NaN,0.515,0.496,10453.12,10453.12,16507.85,16507.85
2,10002,New,Female,75-79,No,Non,Aged,No,False,True,True,85.0,Congestive Heart Failure,0.860,0.829,6455.06,6455.06,6583.45,6583.45
3,10004,New,Female,80-84,No,Non,Aged,No,False,False,True,108.0,Vascular Disease,1.014,0.977,20881.06,20881.06,15210.91,15210.91
4,10007,New,Female,60-64,No,Non,Disabled,No,False,True,True,NaN,NaN,0.000,0.000,28878.83,28878.83,16941.26,16941.26


Summary stats

In [93]:
# corr
df.loc[~df.hcc_code.isna(),['hcc_code','total_paid','total_allowed','normalized_risk_score']].corr().round(2)

,hcc_code,total_paid,total_allowed,normalized_risk_score
hcc_code,1.00,0.11,0.03,0.12
total_paid,0.11,1.00,0.58,-0.01
total_allowed,0.03,0.58,1.00,0.11
normalized_risk_score,0.12,-0.01,0.11,1.00


In [97]:
df.total_paid.describe()

count      1127.000000
mean      13630.763815
std       17269.928951
min           0.000000
25%        2663.775000
50%        7743.590000
75%       18344.270000
max      165506.630000
Name: total_paid, dtype: float64

In [125]:
# distribution of cost by quartiles (avg patient should cost $10k / yr)
df['quartile'] = (pd.qcut(df['total_paid'].values, 4, labels=[1,2,3,4]))
df.groupby('quartile')['total_paid'].agg(['min','mean','max']).round(2)

,min,mean,max
quartile,,,
1,0.00,838.26,2655.67
2,2671.88,5030.92,7743.59
3,7812.04,12620.95,18314.44
4,18374.10,36029.34,165506.63


In [117]:
# hccs by quartile
l = [
    'Congestive Heart Failure', 
    'Acute Myocardial Infarction',
    'Breast, Prostate, and Other Cancers and Tumors',
    'Acute Renal Failure',
    'Specified Heart Arrhythmias','Diabetes without Complication',
    'Rheumatoid Arthritis and Inflammatory Connective Tissue Disease',
    'Major Depressive, Bipolar, and Paranoid Disorders'
]

df.loc[df.description.isin(l)].groupby(['description','quartile']).size()

description                                                      quartile
Acute Myocardial Infarction                                      1            0
                                                                 2            3
                                                                 3            1
                                                                 4            9
Acute Renal Failure                                              1            0
                                                                 2            0
                                                                 3            1
                                                                 4            5
Breast, Prostate, and Other Cancers and Tumors                   1            0
                                                                 2            3
                                                                 3            2
                                              

In [131]:
# if had more patients
# split population into deciles by HCC (hard with 1000 patients)
# find the avg predicted cost in each decile (need prob coef that maps HCCs to predicted cost) / actual total cost

Admissions / Discharge

In [145]:
# encounter
sql = '''
SELECT 
    encounter_id, 
    patient_id, 
    encounter_type,
    admit_source_description,
    admit_type_description,
    discharge_disposition_description,
    paid_amount,
    charge_amount
FROM SANDBOX_CLAIMS.CORE.ENCOUNTER;
'''
enc = read_sql(sql)
enc = enc.sort_values('patient_id').reset_index(drop=True)
enc[['patient_id','paid_amount','charge_amount']] = enc[['patient_id','paid_amount','charge_amount']].apply(pd.to_numeric, errors='coerce')

print(enc.shape)
enc.head()

(200, 8)


,encounter_id,patient_id,encounter_type,admit_source_description,admit_type_description,discharge_disposition_description,paid_amount,charge_amount
0,884077,10004,acute inpatient,Non-Health Care Facility Point of Origin (Phys...,Emergency,Discharged/transferred to home care of organiz...,8033.82,902232.00
1,129259,10010,acute inpatient,Non-Health Care Facility Point of Origin (Phys...,Emergency,Discharged to home/self-care (routine charge),3151.98,244516.86
2,1672699,10013,acute inpatient,Non-Health Care Facility Point of Origin (Phys...,Emergency,Discharged to home/self-care (routine charge),8048.68,362016.00
3,690699,10017,acute inpatient,Transfer from Skilled Nursing Facility (SNF),Emergency,Left against medical advice or discontinued care.,6786.55,118169.76
4,887368,10033,acute inpatient,Transfer from Hospital (different facility),Urgent,Discharged/transferred to skilled nursing faci...,12846.91,391537.50


In [147]:
# discharge info
# for post acute patients (discharged from admission to somewhere else like skilled nursing). Makes things expensive. 

l = [
    'Discharged to home/self-care (routine charge)',
    'Discharged/transferred to an inpatient rehabilitation facility including distinct parts units of a hospital.',
    'Discharged/transferred to skilled nursing facility (SNF) with Medicare certification in anticipation of covered skilled care.',
]
enc.loc[enc['discharge_disposition_description'].isin(l),:].groupby('discharge_disposition_description')['paid_amount'].agg(['min','mean','max']).round(2)

,min,mean,max
discharge_disposition_description,,,
Discharged to home/self-care (routine charge),0.00,9336.93,136818.52
Discharged/transferred to an inpatient rehabilitation facility including distinct parts units of a hospital.,3457.10,13034.27,43228.49
Discharged/transferred to skilled nursing facility (SNF) with Medicare certification in anticipation of covered skilled care.,823.82,10833.68,44705.14


In [148]:
enc.loc[enc['paid_amount']>130000.,:]

,encounter_id,patient_id,encounter_type,admit_source_description,admit_type_description,discharge_disposition_description,paid_amount,charge_amount
11,1296604,10124,acute inpatient,Non-Health Care Facility Point of Origin (Phys...,Elective,Discharged to home/self-care (routine charge),136818.52,1042120.8


In [174]:
# admission info
# look for expensive patients (do they have admissions)
enc['quartile'] = (pd.qcut(enc['paid_amount'].values, 4, labels=[1,2,3,4]))
enc.head()

res = pd.DataFrame(enc.groupby('admit_source_description')['quartile'].value_counts()).sort_values(['admit_source_description','quartile'])
res

count
admit_source_description                           quartile       
Clinic Referral                                    1             4
                                                   2             3
                                                   3             3
                                                   4             7
Non-Health Care Facility Point of Origin (Physi... 1            45
                                                   2            44
                                                   3            43
                                                   4            41
Transfer from Hospital (different facility)        1             0
                                                   2             1
                                                   3             2
                                                   4             2
Transfer from Skilled Nursing Facility (SNF)       1             1
                                                   2             2
                                                   3             2
                                                   4             0

HCC category to predicted cost

In [ ]:
# to do -- not enough data?

In [35]:
close_connection()